In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
# school_data
# student_data

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# school_data_complete_df.head()

# District Summary

In [2]:
studentCount = school_data_complete_df["student_name"].count()
schoolsList = school_data_complete_df["school_name"].unique()
schoolCount = int(len(schoolsList))
readAvg = school_data_complete_df["reading_score"].mean()
mathAvg = school_data_complete_df["math_score"].mean()
mathPassing = school_data_complete_df.loc[school_data_complete_df["math_score"] >= 70]
readPassing = school_data_complete_df.loc[school_data_complete_df["reading_score"] >= 70]
overallPassing = school_data_complete_df.loc[(school_data_complete_df["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]


totalBudget = school_data["budget"].sum()
mathPercent = (len(mathPassing)/studentCount)*100
readPercent = (len(readPassing)/studentCount)*100
overallPercent = (len(overallPassing)/studentCount)*100
# mathPercent
# readPercent
# overallPercent

districtSummary_df = pd.DataFrame({"Total Schools": [schoolCount],
                                   "Total Students":[studentCount],
                                   "Total Budget": totalBudget,
                                   "Average Math Score": [mathAvg], 
                                   "Average Reading Score": [readAvg], 
                                   "% Passing Math": mathPercent,
                                   "% Passing Reading":readPercent, 
                                   "% Overall Passing":overallPercent})

districtSummary_df["Total Students"] = districtSummary_df["Total Students"].map("{:,}".format)
districtSummary_df["Total Budget"] = districtSummary_df["Total Budget"].map("${:,}".format)
districtSummary_df["Average Math Score"] = districtSummary_df["Average Math Score"].map("{:.2f}".format)
districtSummary_df["Average Reading Score"] = districtSummary_df["Average Reading Score"].map("{:.2f}".format)
districtSummary_df["% Passing Math"] = districtSummary_df["% Passing Math"].map("{:.2f}%".format)
districtSummary_df["% Passing Reading"] = districtSummary_df["% Passing Reading"].map("{:.2f}%".format)
districtSummary_df["% Overall Passing"] = districtSummary_df["% Overall Passing"].map("{:.2f}%".format)
districtSummary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


# School Summary

In [3]:
reduced_school_data = school_data[["school_name","type","size","budget"]]
schoolGroup_df = reduced_school_data.set_index("school_name").sort_values("school_name",ascending = True)
studentBudget = schoolGroup_df["budget"]/schoolGroup_df["size"]
schoolGroup_df["Per Student Budget"] = studentBudget
# schoolGroup_df

In [4]:
#Average columns for math and reading scores
reduced_student_data = student_data[["school_name", "math_score", "reading_score"]]
scoreGroup_df = reduced_student_data.groupby("school_name").mean()
scoreGroup_df

#Percentages columns for math, reading and overall
##Find passing students
mathSchoolPassing = reduced_student_data.loc[reduced_student_data["math_score"] >= 70]
readSchoolPassing = reduced_student_data.loc[reduced_student_data["reading_score"] >= 70]
overallSchoolPassing = reduced_student_data.loc[(reduced_student_data["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]

## Calculating Passing DFs 
mathCounts = mathSchoolPassing.groupby("school_name").count()
mathSchoolPercent = (mathCounts["math_score"]/schoolGroup_df["size"])*100

readingCounts = readSchoolPassing.groupby("school_name").count()
readSchoolPercent = (readingCounts["reading_score"]/schoolGroup_df["size"])*100

allCounts = overallSchoolPassing.groupby("school_name").count()
overallSchoolPercent = (allCounts["reading_score"]/schoolGroup_df["size"])*100

passPercentages_df = pd.DataFrame({"% Passing Math": mathSchoolPercent,
                                   "% Passing Reading":readSchoolPercent,
                                   "% Overall Passing": overallSchoolPercent,
                                  })
passPercentages_df

partialSummary_df = pd.merge(schoolGroup_df,scoreGroup_df, on="school_name")
partialSummary_df

schoolSummary_df = pd.merge(partialSummary_df,passPercentages_df, on="school_name")
schoolSummary_df

renamed_schoolSummary_df = schoolSummary_df.rename(columns={"type":"School Type", 
                                                            "size":"Total Students",
                                                            "budget":"Total School Budget",
                                                            "math_score":"Average Math Score",
                                                            "reading_score":"Average Reading Score"
                                                           })
renamedF_schoolSummary_df = renamed_schoolSummary_df.copy()

renamedF_schoolSummary_df["Total Students"] = renamedF_schoolSummary_df["Total Students"].map("{:,}".format)
renamedF_schoolSummary_df["Total School Budget"] = renamedF_schoolSummary_df["Total School Budget"].map("${:,}".format)
renamedF_schoolSummary_df["Per Student Budget"] = renamedF_schoolSummary_df["Per Student Budget"].map("${:,.0f}".format)
renamedF_schoolSummary_df["Average Math Score"] = renamedF_schoolSummary_df["Average Math Score"].map("{:.2f}".format)
renamedF_schoolSummary_df["Average Reading Score"] = renamedF_schoolSummary_df["Average Reading Score"].map("{:.2f}".format)
renamedF_schoolSummary_df["% Passing Math"] = renamedF_schoolSummary_df["% Passing Math"].map("{:.2f}%".format)
renamedF_schoolSummary_df["% Passing Reading"] = renamedF_schoolSummary_df["% Passing Reading"].map("{:.2f}%".format)
renamedF_schoolSummary_df["% Overall Passing"] = renamedF_schoolSummary_df["% Overall Passing"].map("{:.2f}%".format)
renamedF_schoolSummary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


# Top Performing Schools (By % Overall Passing)

In [21]:
top_renamed_schoolSummary_df = renamedF_schoolSummary_df.sort_values("% Overall Passing", ascending=False)
top_renamed_schoolSummary_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


# Bottom Performing Schools (By % Overall Passing)


In [22]:
top_renamed_schoolSummary_df = renamedF_schoolSummary_df.sort_values("% Overall Passing", ascending=True)
top_renamed_schoolSummary_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


In [7]:
gradeNine = {}
gradeTen = {}
gradeEleven = {}
gradeTwelve = {}

schoolsNine = []
mathScoreNine = []
readScoreNine = []
schoolsTen = []
mathScoreTen = []
readScoreTen = []
schoolsEleven = []
mathScoreEleven = []
readScoreEleven = []
schoolsTwelve = []
mathScoreTwelve = []
readScoreTwelve = []

#Separate scores by school and grade
for i in range(studentCount) :
    if student_data["grade"][i] == "9th":
        schoolsNine.append(student_data["school_name"][i])
        readScoreNine.append(student_data["reading_score"][i])
        mathScoreNine.append(student_data["math_score"][i])
        gradeNine = {"School Name": schoolsNine,
                    "Reading Score": readScoreNine,
                    "Math Score": mathScoreNine
                   } 
    elif student_data["grade"][i] == "10th":
        schoolsTen.append(student_data["school_name"][i])
        readScoreTen.append(student_data["reading_score"][i])
        mathScoreTen.append(student_data["math_score"][i])
        gradeTen = {"School Name": schoolsTen,
                    "Reading Score": readScoreTen,
                    "Math Score": mathScoreTen
                   }
    elif student_data["grade"][i] == "11th":
        schoolsEleven.append(student_data["school_name"][i])
        readScoreEleven.append(student_data["reading_score"][i])
        mathScoreEleven.append(student_data["math_score"][i])
        gradeEleven = {"School Name": schoolsEleven,
                    "Reading Score": readScoreEleven,
                    "Math Score": mathScoreEleven
                   }
    else: 
        schoolsTwelve.append(student_data["school_name"][i])
        readScoreTwelve.append(student_data["reading_score"][i])
        mathScoreTwelve.append(student_data["math_score"][i])
        gradeTwelve = {"School Name": schoolsTwelve,
                    "Reading Score": readScoreTwelve,
                    "Math Score": mathScoreTwelve
                   }
# Create grade dataframes and merge them all together
GN_df = pd.DataFrame(gradeNine)
renamed_GN_df = GN_df.rename(columns={"Reading Score":"9th R",
                                      "Math Score":"9th M"})
gradeNineGroup_df = renamed_GN_df.groupby("School Name").mean()

GT_df = pd.DataFrame(gradeTen)
renamed_GT_df = GT_df.rename(columns={"Reading Score":"10th R",
                                      "Math Score":"10th M"})
gradeTenGroup_df = renamed_GT_df.groupby("School Name").mean()

GE_df = pd.DataFrame(gradeEleven)
renamed_GE_df = GE_df.rename(columns={"Reading Score":"11th R",
                                      "Math Score":"11th M"})
gradeElevenGroup_df = renamed_GE_df.groupby("School Name").mean()

GTw_df = pd.DataFrame(gradeTwelve)
renamed_GTw_df = GTw_df.rename(columns={"Reading Score":"12th R",
                                      "Math Score":"12th M"})
gradeTwelveGroup_df = renamed_GTw_df.groupby("School Name").mean()

partialScores1_df = pd.merge(gradeNineGroup_df,gradeTenGroup_df, on="School Name")
partialScores2_df = pd.merge(partialScores1_df,gradeElevenGroup_df, on="School Name")
partialScoresF_df = pd.merge(partialScores2_df,gradeTwelveGroup_df, on="School Name")
# partialScoresF_df

# Math Scores by Grade

In [8]:
mathScores_df = partialScoresF_df[["9th M","10th M","11th M","12th M"]]
renamed_mathScores_df = mathScores_df.rename(columns = {"9th M": "9th","10th M":"10th","11th M":"11th","12th M":"12th"})


renamed_mathScores_df["9th"] = renamed_mathScores_df["9th"].map("{:.2f}".format)
renamed_mathScores_df["10th"] = renamed_mathScores_df["10th"].map("{:.2f}".format)
renamed_mathScores_df["11th"] = renamed_mathScores_df["11th"].map("{:.2f}".format)
renamed_mathScores_df["12th"] = renamed_mathScores_df["12th"].map("{:.2f}".format)
renamed_mathScores_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


# Reading Score by Grade 

In [9]:
readScores_df = partialScoresF_df[["9th R","10th R","11th R","12th R"]]
renamed_readScores_df = readScores_df.rename(columns = {"9th R": "9th","10th R":"10th","11th R":"11th","12th R":"12th"})

renamed_readScores_df["9th"] = renamed_readScores_df["9th"].map("{:.2f}".format)
renamed_readScores_df["10th"] = renamed_readScores_df["10th"].map("{:.2f}".format)
renamed_readScores_df["11th"] = renamed_readScores_df["11th"].map("{:.2f}".format)
renamed_readScores_df["12th"] = renamed_readScores_df["12th"].map("{:.2f}".format)
renamed_readScores_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


# Scores by School Spending

In [16]:
schoolSpend_df = renamed_schoolSummary_df

bins= [0,585,630,645,680]
groupNames = ["<$585","$585-630","$630-645","$645-680"]

schoolSpend_df["Spending Ranges (Per Student)"] = pd.cut(schoolSpend_df["Per Student Budget"], bins,labels=groupNames, include_lowest=True)
spendSummary_df = schoolSpend_df[["Spending Ranges (Per Student)","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
spendSummaryF_df = spendSummary_df.groupby("Spending Ranges (Per Student)").mean()

#Formatting data
spendSummaryF_df.reset_index
spendSummaryF_df["Average Math Score"] = spendSummaryF_df["Average Math Score"].map("{:.2f}".format)
spendSummaryF_df["Average Reading Score"] = spendSummaryF_df["Average Reading Score"].map("{:.2f}".format)
spendSummaryF_df["% Passing Math"] = spendSummaryF_df["% Passing Math"].map("{:.2f}%".format)
spendSummaryF_df["% Passing Reading"] = spendSummaryF_df["% Passing Reading"].map("{:.2f}%".format)
spendSummaryF_df["% Overall Passing"] = spendSummaryF_df["% Overall Passing"].map("{:.2f}%".format)
spendSummaryF_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


# Scores by School Size

In [18]:
schoolSize_df = renamed_schoolSummary_df.copy()

bins2 = [0,1000,2000,5000]
groupNames2 = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

schoolSize_df["School Size"] = pd.cut(schoolSize_df["Total Students"], bins2,labels=groupNames2, include_lowest=True)
sizeSummary_df = schoolSize_df[["School Size","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
sizeSummaryF_df = sizeSummary_df.groupby("School Size").mean()

#Formatting data
sizeSummaryF_df.reset_index
sizeSummaryF_df["Average Math Score"] = sizeSummaryF_df["Average Math Score"].map("{:.2f}".format)
sizeSummaryF_df["Average Reading Score"] = sizeSummaryF_df["Average Reading Score"].map("{:.2f}".format)
sizeSummaryF_df["% Passing Math"] = sizeSummaryF_df["% Passing Math"].map("{:.2f}%".format)
sizeSummaryF_df["% Passing Reading"] = sizeSummaryF_df["% Passing Reading"].map("{:.2f}%".format)
sizeSummaryF_df["% Overall Passing"] = sizeSummaryF_df["% Overall Passing"].map("{:.2f}%".format)
sizeSummaryF_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


# Scores by School Type

In [20]:
schoolType_df = renamed_schoolSummary_df.copy()
reduced_schoolType_df = schoolType_df[["School Type","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
typeSummaryF_df = reduced_schoolType_df.groupby("School Type").mean()

#Formatting data
typeSummaryF_df.reset_index
typeSummaryF_df["Average Math Score"] = typeSummaryF_df["Average Math Score"].map("{:.2f}".format)
typeSummaryF_df["Average Reading Score"] = typeSummaryF_df["Average Reading Score"].map("{:.2f}".format)
typeSummaryF_df["% Passing Math"] = typeSummaryF_df["% Passing Math"].map("{:.2f}%".format)
typeSummaryF_df["% Passing Reading"] = typeSummaryF_df["% Passing Reading"].map("{:.2f}%".format)
typeSummaryF_df["% Overall Passing"] = typeSummaryF_df["% Overall Passing"].map("{:.2f}%".format)
typeSummaryF_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
